In [1]:
import pandas as pd
import numpy as np
import sys
from pyspark.sql import functions as f
import re
from functools import reduce
from pyspark.sql import SparkSession
from typing import Dict
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import re
from pyspark.sql.window import Window
import plotly.express as px
from pyspark.ml.feature import Bucketizer
from itertools import chain
from pandas.tseries.offsets import MonthEnd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

In [2]:
def run_spark_session(name: str):
    spark = SparkSession.builder.master('yarn').appName(f'{name}').getOrCreate()
    spark.sparkContext.setLogLevel('OFF')
    spark.conf.set("spark.sql.session.timeZone","GMT")
    spark.conf.set('spark.sql.execution.arrow.enabled', 'False')
    spark.conf.set('spark.sql.crossJoin.enabled', 'true')
    return spark

#spark = run_spark_session('collect_')

## Сборка таблицы

In [3]:
def get_most_freq(df, groupby, columns):
    for col in columns:
        w = Window.partitionBy(groupby)
        df = df.withColumn(f'{col}_count', f.count(col).over(w))
        df = df.withColumn(f'{col}_count_rank', f.row_number().over(w.orderBy(f.col(f'{col}_count').desc())))
        df = df.withColumn(col, f.when(f.col(f'{col}_count_rank') == 1, f.col(col)).otherwise(None))
        df = df.drop(f'{col}_count', f'{col}_count_rank')
    return df        

def get_escrow():
    columns = ['complexId', 'developerId','groupId',
               'address_additionalInfo','address_building','address_district','address_domain','address_house','address_letter',
               'address_localityName', 'address_localityType','address_region_code','address_region_name',
               'address_streetName', 'address_streetType','address_unit','addressString',
               'buildRegion_code','buildRegion_name',
               'latitude', 'longitude', 'livingArea', 'objectStatus_code', 'objectStatus_name', 'projectDeclaration_number']
    
    df = get_most_freq(spark.table('prod_domrf.domrf_object'), ['objectid', 'commissioningPlanDate'], columns)
                           
    df = df.groupby('objectid', 'commissioningPlanDate').agg(*[f.first(col).alias(col) for col in columns])
    
    
    df_bp = get_most_freq(spark.table('prod_domrf.domrf_object_buildpermit'), ['objectid', 'number'], ['issuedate', 'issuedby'])
    df_bp = df_bp.groupby('objectid', 'number').agg(f.first('issuedate').alias('issuedate'), f.max('renewaldate').alias('renewaldate'), f.first('issuedby').alias('issuedby'))
    
    df = df.join(df_bp, on = (df['objectid'] == df_bp['objectid']) & (df['commissioningPlanDate'] >= df_bp['issuedate']) ).drop(df_bp['objectid'])

    df = get_most_freq(df, ['objectid', 'number'], [*columns, 'commissioningPlanDate', 'issuedate', 'issuedby']).groupby('objectid', 'number').agg(
                                                            *[f.first(col).alias(col) for col in [*columns, 'commissioningPlanDate', 'issuedate', 'issuedby']])
    
    return df               
                           
    

In [3]:
def get_market_rates(spark):
    rates_config_list = [
        {
            'query_filter': {'index_group_name': 'MOSPRIME%'}
        }  ,
        {
            'query_filter': {'type_name_eng': 'Key rate of CBR on every workingday basis'}
        },
        {
            'query_filter': {'type_name_eng': 'Russia Money Supply M2%', 'index_group_name': '%Денежный агрегат M2 (ежемесячная периодичность)%'}
        },
        {
            'query_filter': {'type_name_eng': 'MOEXREPO%'},
            'proc_function': lambda df: df[["MOEXREPO 1 week 12:30", "MOEXREPO 1 week 19:00", "MOEXREPO eq 12:30", "MOEXREPO eq 19:00", "MOEXREPO obl 19:00"]]
        },
        {
            'query_filter': {'type_name_eng': 'RUONIA%'}
        }
    ]
    df = spark.table('prod_etl_cbonds.cbonds_index_value')
    rates_list = []
    for config in rates_config_list:
        rate_df = df.filter(reduce(lambda c1, c2: c1 & c2, [f.col(col).like(like) for col, like in config['query_filter'].items()]) ).select('date', 'type_name_eng', 'value')
        rate_df = rate_df.toPandas().pivot_table(index = 'date', columns = 'type_name_eng', values = 'value', aggfunc = 'mean')
        if config.get('proc_function'):
            rate_df = config['proc_function'](rate_df)
            
        #rate_df.index = pd.to_datetime(rate_df.index)
        rate_df = rate_df.resample('M').apply(lambda x: x.mean())
        rates_list.append(rate_df)
    
    market_rates = pd.concat(rates_list, axis = 1)
    market_rates = market_rates[market_rates.index >= '2014-01-01']
    market_rates.index.name = 'report_date'
    return market_rates
mr = get_market_rates(spark)

In [4]:
mr

type_name_eng,MOSPRIME 1M,MOSPRIME 1W,MOSPRIME 2M,MOSPRIME 2W,MOSPRIME 3M,MOSPRIME 6M,MOSPRIME O/N,Key rate of CBR on every workingday basis,Russia Money Supply M2,MOEXREPO 1 week 12:30,MOEXREPO 1 week 19:00,MOEXREPO eq 12:30,MOEXREPO eq 19:00,MOEXREPO obl 19:00,RUONIA 1M,RUONIA 3M,RUONIA 6M,RUONIA Consensus Forecasts Q1 2024,RUONIA Consensus Forecasts Q1 2025,RUONIA Consensus Forecasts Q2 2024,RUONIA Consensus Forecasts Q3 2024,RUONIA Consensus Forecasts Q4 2024,RUONIA Index,RUONIA rate
report_date,,,,,,,,,,,,,,,,,,,,,,,,
2014-01-31,6.730000,6.478235,6.883529,6.544706,7.042941,7.235294,6.344118,5.500000,29953.433333,NaN,NaN,NaN,NaN,NaN,6.276129,6.251613,6.216774,NaN,NaN,NaN,NaN,NaN,1.200626,6.047059
2014-02-28,6.773000,6.426000,6.962000,6.524000,7.115500,7.289500,6.228000,5.500000,30266.600000,NaN,NaN,NaN,NaN,NaN,5.989286,6.221429,6.208571,NaN,NaN,NaN,NaN,NaN,1.206433,5.963500
2014-03-31,8.736000,8.213000,8.886000,8.333500,9.005000,9.122000,8.074000,7.000000,29613.633333,NaN,NaN,NaN,NaN,NaN,6.841290,6.389032,6.333548,NaN,NaN,NaN,NaN,NaN,1.213073,7.830500
2014-04-30,8.872727,8.314545,9.078636,8.453182,9.279091,9.415909,8.108182,7.090909,29973.333333,NaN,NaN,NaN,NaN,NaN,7.803333,6.910000,6.630667,NaN,NaN,NaN,NaN,NaN,1.220981,7.821364
2014-05-31,9.324211,8.704211,9.581053,8.910000,9.802105,9.928947,8.415789,7.500000,30009.700000,NaN,NaN,NaN,NaN,NaN,8.061613,7.634194,6.977097,NaN,NaN,NaN,NaN,NaN,1.229205,8.152632
2014-06-30,9.196842,8.545263,9.357895,8.723684,9.570000,9.791053,8.396316,7.500000,30191.200000,NaN,NaN,NaN,NaN,NaN,8.096333,8.038667,7.282333,NaN,NaN,NaN,NaN,NaN,1.237521,8.182105
2014-07-31,9.217391,8.664348,9.386957,8.814783,9.557391,9.824783,8.430435,7.608696,30294.400000,NaN,NaN,NaN,NaN,NaN,8.143548,8.156452,7.611613,NaN,NaN,NaN,NaN,NaN,1.245946,8.123043
2014-08-31,9.521429,8.575714,9.853333,8.902857,10.154762,10.311429,8.138571,8.000000,30456.633333,NaN,NaN,NaN,NaN,NaN,8.007419,8.137742,7.966452,NaN,NaN,NaN,NaN,NaN,1.254419,7.851905
2014-09-30,9.447273,8.622273,9.847727,8.861818,10.282273,10.441364,8.198636,8.000000,30413.933333,NaN,NaN,NaN,NaN,NaN,7.936000,8.083000,8.144333,NaN,NaN,NaN,NaN,NaN,1.262736,7.944091


In [4]:
esc = get_escrow().toPandas()

In [5]:
esc.to_parquet('escrow_base.parquet')

In [2]:
esc = pd.read_parquet('escrow_base.parquet')

In [3]:
esc.groupby(['objectid', 'number']).count().max(axis = 1).max()

1

In [15]:
#df = pd.read_excel('EscrowReport_All_07.03.2024.xlsx').iloc[3:, :]
#df.columns = df.iloc[0, :].values
#df = df.iloc[1:, :]
df = pd.read_excel('EscrowReport_All_20.02.2024.xlsx').iloc[1:]
df.columns = df.iloc[0, :].values
df = df.iloc[1:, 1:]

In [16]:
df

,Дата открытия счета,Плановая дата окончания,Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Текущий остаток по счету,в т.ч. собственные средства,в т.ч. ипотечные средства,Наименование получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Адрес объекта недвижимости,"Площадь объекта ДДУ, м2",Номер разрешения на строительство,Номер корпуса объекта ДДУ,Сумма по ДДУ,Сегмент застройщика,ОГРН застройщика,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Дата выдачи разрешения на строительство,"Баланс денежных средств на счете, согласно ДДУ","Баланс собственных денежных средств на счете, согласно ДДУ","Баланс ипотечных средств ВТБ на счете, согласно ДДУ","Баланс ипотечных средств другого банка на счете, согласно ДДУ","Баланс средств материнского капитала на счете, согласно ДДУ","Баланс средств военной ипотеки на счете, согласно ДДУ","Баланс прочих средств на счете, согласно ДДУ",Баланс ипотечных средств другого банка на счете,Баланс средств материнского капитала на счете,Баланс средств военной ипотеки на счете,Баланс прочих средств на счете,Последняя дата пополнения счета,Последняя сумма пополнения счета,Фактическая дата закрытия счета,Владелец счета,Назначение помещения
2,2019-07-30 00:00:00,2019-07-30 00:00:00,0,3103,Закрыт,40824810831036000011,0.0,0.0,0.0,-,-,-,-,-,-,-,-,NaN,-,0.0,-,-,-,Нет,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-30 00:00:00,-,2019-07-30 00:00:00,ФЛ,Нежилое помещение
3,2019-07-31 00:00:00,2019-07-31 00:00:00,0,3103,Закрыт,40824810231036000019,0.0,0.0,0.0,-,-,-,-,-,-,-,-,NaN,-,0.0,-,-,-,Нет,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-31 00:00:00,-,2019-07-31 00:00:00,ФЛ,Нежилое помещение
4,2019-08-01 00:00:00,2019-08-01 00:00:00,0,3103,Закрыт,40824810131036000025,0.0,0.0,0.0,-,-,-,-,-,-,-,-,NaN,-,0.0,-,-,-,Нет,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-08-01 00:00:00,-,2019-08-01 00:00:00,ФЛ,Нежилое помещение
5,2019-08-02 00:00:00,2019-08-02 00:00:00,0,1025,Закрыт,40824810010256000010,0.0,0.0,0.0,-,-,-,-,-,-,-,-,NaN,-,0.0,-,-,-,Нет,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-08-02 00:00:00,-,2019-08-02 00:00:00,ФЛ,Нежилое помещение
6,2019-08-02 00:00:00,2019-08-02 00:00:00,0,3103,Закрыт,40824810031036000028,0.0,0.0,0.0,-,-,-,-,-,-,-,-,NaN,-,0.0,-,-,-,Нет,-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-08-02 00:00:00,-,2019-08-02 00:00:00,ФЛ,Нежилое помещение
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358039,2023-12-19 00:00:00,2025-11-10 00:00:00,001,1025,Закрыт,40824810110258001454,0.0,0.0,0.0,"ООО ""СЗ ""СЕРДЦЕ СТОЛИЦЫ""",9729299057,044525187,40702810106800002625,СС-19945ИП,2023-12-11 00:00:00,"Квартира, город Москва, вн.тер.г. муниципальны...",68.2,77-212000-019692-2021,Объект: 1,29639506.0,КИБ,1207700294801,2023-12-11 00:00:00,Нет,2021-09-04 00:00:00,29639506.0,0.0,29639506.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-12-21 00:00:00,-29639506.0,2023-12-21 00:00:00,ФЛ,Жилое помещение
358040,2023-12-20 00:00:00,2025-03-30 00:00:00,040,2443,На подтверждении,40824810624438000005,0.0,0.0,0.0,"ООО ""СЗ ""СЕРДЦЕ СТОЛИЦЫ""",9729299057,044525187,40702810106800002625,СС-19974Г,2023-12-18 00:00:00,"город Москва, вн.тер.г. муниципальный округ Хо...",13.3,77-212000-019692-2021,Объект: 1,2960000.0,КИБ,1207700294801,2023-12-18 00:00:00,Нет,2021-08-04 00:00:00,2960000.0,2960000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,-,ФЛ,Нежилое помещение
358041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def merge_dfs(objects, assets):
    assets_patterns_to_del = [
        '\s',
        '-',
        '[a-zA-Zа-яА-ЯёЁ]',
        '/\d{1,3}\b',
        '/',
        '\(.*\)',
        '\D',
    ]
    
    object_patterns_to_del = [
       '\s',
        '-',
        '[a-zA-Zа-яА-ЯёЁ]',
        '/\d{1,3}\b',
        '/',
        '\(.*\)',
        '\b[\(:;,№].{5,}',
        '\b["].{7,}',
        '\D', 
    ]
    
    object_patterns_to_del = r'\s|-|[a-zA-Zа-яА-яёЁ]|\(.*\)|/\d{1,3}\Z|\b[\(:;,№].{5,}|\b["].{7,}|\D'
    assets_patterns_to_del = r'\s|-|[a-zA-Zа-яА-яёЁ]|\(.*\)|/\d{1,3}\Z|\D'
    
    merge = lambda df1, df2, patt1, patt2: df1.assign(number_clean = df1['number'].map(
            lambda x: re.sub(patt1, '', x) if x is not None else None)).groupby(['objectid', 'number_clean'], as_index = False).agg(
            lambda df: df if (df.shape[0] < 2) else df.mode()[0] if len(df.mode()) > 0 else None).merge(
        
        df2.assign(number_clean = df2['Номер разрешения на строительство'].map(
                    lambda x: re.sub(patt2, '', str(x)) if x is not None else None)),
                    on = 'number_clean', how = 'inner'
    )
    
    df = merge(objects, assets, object_patterns_to_del, assets_patterns_to_del)
    
    for obj_patts, assets_patts in zip(['\A.{2,5}[a-zA-Zа-яА-яёЁ:;"]', '\A.{2,5}[a-zA-Zа-яА-яёЁ:;"]', ''],
                                       ['\A.{2,5}[a-zA-Zа-яА-яёЁ:;"]', '', '\A.{2,5}[a-zA-Zа-яА-яёЁ:;"]']):
    
        add_df = merge(objects, assets[~assets['Номер счета эскроу'].isin(df['Номер счета эскроу'].unique())], 
                      '|'.join([obj_patts, object_patterns_to_del]), '|'.join([assets_patts, assets_patterns_to_del]))

        df = pd.concat([df, add_df])
    
    
    
    
    #return df
    #return objects[~objects['number'].isin(df['number'].unique())]#assets[~assets['Номер счета эскроу'].isin(df['Номер счета эскроу'].unique())]
    #return merge(objects[~objects['number'].isin(df['number'].unique())], assets[~assets['Номер счета эскроу'].isin(df['Номер счета эскроу'].unique())], 
    #                 r'\s|-|\b[а-яА-яёЁ]{2,}.*|\(.*\)|/\d{1,3}\Z|\D', r'\s|-|[a-zA-Zа-яА-яёЁ]|\(.*\)|\D')
    
    
    # Плохие объекты - для которых несколько номеров рнс. Обычно они идентичные, дело в обработке строк номеров. Можно просто заполнить модой 
    bad_objectid = df.groupby(['objectid']).apply(lambda df: df['number_clean'].nunique())
    bad_objectid = bad_objectid[bad_objectid > 1].index
    print(df['Номер счета эскроу'].nunique())
    df = df[~df['number_clean'].isin([''])]
    print(df['Номер счета эскроу'].nunique())
    
    df = df.groupby(['objectid'], as_index = False).apply(lambda df: df.assign(
        **{col: df[col].mode()[0] if len(df[col].mode()) > 0 else None for col in ['Номер разрешения на строительство', 'number_clean', *esc.columns]})).reset_index(drop = True)
    
    print('2', (df['Номер разрешения на строительство'].map(lambda x: re.sub(assets_patterns_to_del, '', str(x)) if x is not None else None) != df['number_clean']).sum())
    # Далее плохие номера РНС, для которых характерно несколько объектов, из-за этого не получается определить, к какому именно объекту относится счет, 
    # потому что мерж происходит только по номеру РНС. 
    # Простой вариант - считать что все относится к одному объекту, для которого больше всего счетов. Так как теперь у нас для каждого объекта 1 номер РНС,
    # достаточно просто срезать самый частый объект по партиции
    print(df['Номер счета эскроу'].nunique())
    
    df = df.groupby(['number_clean'], as_index = False).apply(lambda df: df.assign(**{col: df[col].agg(f) for col, f in 
                                zip(['commissioningPlanDate', 'issuedate'], ['max', 'min'])}).assign(
                    LivingArea = df.groupby(['objectid']).agg({'livingArea': 'first'}).iloc[:, 0].sum())[df['objectid'] == df['objectid'].mode()[0]]).reset_index(drop = True)
    print('3', (df['Номер разрешения на строительство'].map(lambda x: re.sub(assets_patterns_to_del, '', str(x)) if x is not None else None) != df['number_clean']).sum())
    
    #df = df.groupby(['Номер счета эскроу'], as_index = False).apply(lambda df: df[df['Номер разрешения на строительство'].map(lambda x:
    #                            re.sub(assets_patterns_to_del, '', str(x)) if x is not None else None) == df['number_clean']]).reset_index(drop = True)

    return df 
dff = merge_dfs(esc, df)

269961
269935
2 215
269935
3 215


In [24]:
dff['objectid'].agg('max')

46217

In [19]:
df[~df['Номер счета эскроу'].isin(dff['Номер счета эскроу'].unique())]['Номер разрешения на строительство'].unique()

array([nan, '-', '00-000000-000000-2019', '0000000', '123456',
       '23-23/001-23/001/011/2016-2170/2', '01-01:05-19-2023',
       'RU-010530420060001-001', 'RU 010530420060001-001',
       '02-44-21-2022', '02-56-15-2023', '02-56-29-2023',
       '03-03307000-18-2022', '02-RU03308000-1078Ж-2018',
       '02-RU 03308000-1067Ж-2018', '02-RU 03308000-1330Ж-2022',
       '02-RU 03308000-1331Ж-2022', '02-RU03308000-1331Ж-2022',
       '04301000-41-2022', '03-24-13-2023', '03-RU04302000-58-2022',
       '03-24-58-2022', '03-24-58-2022 ', '03-RU04302000-32-2022',
       '03-RU04302000-14-2022', '03-RU04302000-44-2022', '03-24-46-2022',
       '03-24-46-2022 ', '03-24-02-2023', '03-24-07-2023',
       '22-RU22308000-013-2022', '22-69-21-2023',
       '26-RU26308000-107-2018', '26-RU26308000-108-2018',
       '10-01-80-2023', '10-RU10301000-34-2022', '10-01-34-2022',
       '10-01-34-2022 ', '10-01-45-2023', '10-RU10301000-91/3-2021',
       '10-RU10301000-38-2022', '10-01-31-2023', '10-01-1

In [20]:
dff.apply(lambda s: pd.to_numeric(s, errors = 'coerce') if s.dropna().map(lambda x: type(x)).mode()[0] in [float, int] 
            else pd.to_datetime(s, errors = 'coerce') if s.dropna().map(lambda x: type(x)).mode()[0] == datetime 
              else s.astype(s.dropna().map(lambda x: type(x)).mode()[0], errors = 'ignore')).to_parquet('escrow_assets.parquet')

In [18]:
patt1 = r'\s|-|[a-zA-Zа-яА-яёЁ]|\(.*\)|/\d{1,3}\Z|\b[\(:;,№].{5,}|\b["].{7,}|\D'

esc1 = esc.assign(number_clean = esc['number'].map(
            lambda x: re.sub(patt1, '', x) if x is not None else None)).groupby(['objectid', 'number_clean'], as_index = False).agg(
            lambda df: df if (df.shape[0] < 2) else df.mode()[0] if len(df.mode()) > 0 else None)

df1 = df[~df['Номер счета эскроу'].isin(dff['Номер счета эскроу'].unique())]

df1 = df1.assign(number_clean = df1['Номер разрешения на строительство'].map(
                    lambda x: re.sub(r'\s|\A.{2,5}[a-zA-Zа-яА-яёЁ]|\D', '', str(x)) if x is not None else None))

In [20]:
#for num in df1['number_clean'].unique():
#    print('Номер РНС: ', num)
#    filt = esc1[esc1['number_clean'].map(lambda n: num in n)]
#    if filt.shape[0] > 0:
#        print(filt['number'].unique())
#        print(filt['addressString'].unique())
#        print(df1[df1['number_clean'] == num]['Адрес объекта недвижимости (без номера дома/корпуса)'].unique())
#    else:
#        print(':(')
#    

In [37]:
df1

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,NaN,NaN,number_clean


In [27]:
#test[['number_clean', 'number', 'Номер разрешения на строительство', 'Адрес объекта недвижимости (без номера дома/корпуса)', 'addressString']][-1000:-500][['Адрес объекта недвижимости (без номера дома/корпуса)', 'addressString']].values

In [135]:
test.columns

Index([                                                'objectid',
                                                   'number_clean',
                                                         'number',
                                                      'complexId',
                                                    'developerId',
                                                        'groupId',
                                         'address_additionalInfo',
                                               'address_building',
                                               'address_district',
                                                 'address_domain',
                                                  'address_house',
                                                 'address_letter',
                                           'address_localityName',
                                           'address_localityType',
                                            'address_region_co

In [22]:
dff.apply(lambda s: pd.to_numeric(s, errors = 'coerce') if s.dropna().map(lambda x: type(x)).mode()[0] in [float, int] 
            else pd.to_datetime(s, errors = 'coerce') if s.dropna().map(lambda x: type(x)).mode()[0] == datetime 
              else s.astype(s.dropna().map(lambda x: type(x)).mode()[0], errors = 'ignore')).to_parquet('escrow_assets.parquet')

In [15]:
dff.columns[[10, 11, 36]]

Index(['Код Филиала', 'Код ТП', 'Площадь объекта ДДУ, м2'], dtype='object')

In [2]:
dff = pd.read_parquet('escrow_assets.parquet')

In [3]:
c = 0
for num in dff.groupby('Номер счета эскроу').count().max(axis = 1)[(dff.groupby('Номер счета эскроу').count().max(axis = 1) > 1)].index:
    display(dff[dff['Номер счета эскроу'] == num])
    if c > 10:
        pass
    else:
        c+=1

In [33]:
dff

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,number_clean,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
0,2023-10-02,2022-11-15,2023-12-30,2023-10-03,2023-10-02,-2842500.00,-2842500.00,-,-,"ФИЛИАЛ ""ЦЕНТРАЛЬНЫЙ"" БАНКА ВТБ (ПАО)","365 ОО ""РОО ""Калужский""",Закрыт,40202810011430000013,В рамках 214-ФЗ,0.00,0.0,0.0,0.0,0.0,0.00,КС-ЦУ-776602/2022/00008,"ГКУ КО ""УКС""",4027084855,"АО ""СЗ ""КОШЕЛЕВ-ПРОЕКТ""",СМБ,1106312009253,6312102986,044525411,40702810810240001262,0137200001222003738,2022-09-20,2022-10-14,-,"Калужская обл, г Калуга, ул Андрея Алешина, д....",-,37.90,RU40-301000-006-2022,2022-03-17,2842500.00,0.0,0.0,0.0,0.0,0.0,-,-,ЮЛ,403010000062022,45534,RU40-301000-006-2022,NaN,2893,3.913580e+08,None,None,None,None,124,None,Калуга,г,40,Калужская область,Верховая,улица,None,"обл Калужская, г Калуга, улица Верховая, д. 124",40,Калужская область,54.494171,36.188599,2150.40,1,Строится,40-000352,2023-06-30,2022-03-17,"УПРАВЛЕНИЕ АРХИТЕКТУРЫ, ГРАДОСТРОИТЕЛЬСТВА И З..."
1,2024-02-14,2024-02-02,2025-06-30,NaT,2024-02-14,36149994.20,36149994.20,-,-,"ФИЛИАЛ ""ЦЕНТРАЛЬНЫЙ"" БАНКА ВТБ (ПАО)","ОО ДО ""Московская застава""",Действует,40202810028710000001,В рамках 214-ФЗ,36149994.20,0.0,0.0,0.0,0.0,0.00,КС-2022/89,КИО,7832000076,"ООО ""Эдванс""",СМБ,1027802720945,7805255420,044525411,40702810368800000295,23000810,2024-01-29,2024-02-01,-,"п Шушары, тер Территория предприятия Шушары, д...","Корпус: 10.5, Дом: Участок 312, Объект: 5",213.70,78-016-0422.1-2016,2019-10-21,36149994.20,0.0,0.0,0.0,0.0,0.0,-,-,ЮЛ,78016042212016,24619,78-016-0422.1-2016,NaN,8812,NaN,None,None,None,None,участок 312,None,Шушары,п,78,Санкт-Петербург город,Территория предприятия «Шушары»,None,None,"п Шушары, Территория предприятия «Шушары», д. ...",78,Санкт-Петербург город,59.806874,30.383394,8509.80,1,Строится,78-001234,2022-03-31,2019-10-21,Служба Государственного строительного надзора ...
2,2023-12-28,2022-08-09,2024-06-30,2023-12-28,2023-12-28,-18025087.20,-18025087.20,-,-,"ФИЛИАЛ ""ЦЕНТРАЛЬНЫЙ"" БАНКА ВТБ (ПАО)","ОО ДО ""Московская застава""",Закрыт,40202810028720000004,В рамках 214-ФЗ,0.00,0.0,0.0,0.0,0.0,0.00,КС-2022/49,КИО,7832000076,"ООО ""Эдванс""",СМБ,1027802720945,7805255420,044030704,40702810368800000295,22000253,2022-07-19,2022-08-02,-,"п Шушары, тер Территория предприятия Шушары, д...","Корпус: 10.8, Дом: Участок 312, Объект: 8",120.00,78-016-0422.1-2016,2019-10-21,18025

In [ ]:
def cross_join(df):
    start_date = pd.to_datetime(df['issuedate'].values[0])
    
    end_date = pd.to_datetime(df['Плановая дата окончания'].values[0] if 
            pd.isnull(df['commissioningPlanDate'].values[0]) else df['commissioningPlanDate'].values[0])
    try:
        dates_df = pd.DataFrame({'report_date': pd.date_range(start_date + MonthEnd(0), end_date + MonthEnd(0), freq = 'M')})
        return df.merge(dates_df, how = 'cross')
    except:
        return df


dfff = dff.groupby(['Номер счета эскроу'], as_index = False).apply(lambda df: cross_join(df)).reset_index(drop = True)

In [ ]:
px.line(dfff.groupby(['report_date']).apply(lambda df: df['Общая сумма по ДДУ'].abs().sum()))

In [36]:
dff.to_parquet('escrow_monthly.parquet')

In [283]:
join_df.groupby(['number']).apply(lambda df: df['objectid'].nunique()).sort_values()[-239:]

number
723042032021                    2
36363020000692017               2
36363020000572019               2
36363020000172021               2
57301000702018                  2
                             ... 
52523030002005001062020        39
5252303000200500103012018      48
52523030002005001062021        49
234352392018                   88
                             1320
Length: 239, dtype: int64

In [291]:
join_df[:60]

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,numer,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
0,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3098,,NaN,1224,4.223742e+09,"Микрорайон Королевка, участок ""Старый карьер""",None,Смоленский,None,None,None,Смоленск,город,67,Смоленская область,None,улица,None,"обл Смоленская, Смоленский, город Смоленск, улица",67,Смоленская область,54.812742,32.012638,3490.06,2,Сдан,67-000009,2019-06-30,2017-07-10,Администрация города Смоленска
1,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3727,,NaN,539,1.442112e+09,None,None,None,None,None,None,Ульяновск,город,73,Ульяновская область,None,None,None,"обл Ульяновская, город Ульяновск, Засвияжский",73,Ульяновская область,54.281909,48.261848,NaN,2,Сдан,73-000020,2018-09-30,2016-09-16,Администрация г.Ульяновска
2,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,6464,,NaN,1623,5.155790e+08,None,None,None,None,3,None,Калининград,Город,39,Калининградская область,Александра Невского,Улица,None,"Калининградская область, Город Калининград, Ле...",39,Калининградская область,54.760969,20.541307,2273.60,2,Сдан,39-000173,2019-06-30,2017-11-24,"Агентство по архитектуре, градостроению и перс..."
3,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,13368,,NaN,2028,2.974180e+09,None,None,None,None,None,None,Калининград,город,39,Калининградская область,Карташева,улица,None,"обл Калининградская, город Калининград, улица ...",39,Калининградская область,54.719601,20.373171,NaN,2,Сдан,39-00027

In [360]:
join_df

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,numer,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
0,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3098,,NaN,1224,4.223742e+09,"Микрорайон Королевка, участок ""Старый карьер""",None,Смоленский,None,None,None,Смоленск,город,67,Смоленская область,None,улица,None,"обл Смоленская, Смоленский, город Смоленск, улица",67,Смоленская область,54.812742,32.012638,3490.06,2,Сдан,67-000009,2019-06-30,2017-07-10,Администрация города Смоленска
1,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3727,,NaN,539,1.442112e+09,None,None,None,None,None,None,Ульяновск,город,73,Ульяновская область,None,None,None,"обл Ульяновская, город Ульяновск, Засвияжский",73,Ульяновская область,54.281909,48.261848,NaN,2,Сдан,73-000020,2018-09-30,2016-09-16,Администрация г.Ульяновска
2,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,6464,,NaN,1623,5.155790e+08,None,None,None,None,3,None,Калининград,Город,39,Калининградская область,Александра Невского,Улица,None,"Калининградская область, Город Калининград, Ле...",39,Калининградская область,54.760969,20.541307,2273.60,2,Сдан,39-000173,2019-06-30,2017-11-24,"Агентство по архитектуре, градостроению и перс..."
3,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,13368,,NaN,2028,2.974180e+09,None,None,None,None,None,None,Калининград,город,39,Калининградская область,Карташева,улица,None,"обл Калининградская, город Калининград, улица ...",39,Калининградская область,54.719601,20.373171,NaN,2,Сдан,39-00027

In [363]:
join_df[join_df['number'].isin(join_df.groupby(['number']).apply(lambda df: df['objectid'].nunique()).sort_values()[-239:].index)&
       (join_df['Дата открытия '] <= pd.to_datetime(join_df['commissioningPlanDate']))].groupby(['number']).apply(lambda df: df['objectid'].nunique())

number
                      921
020330800010802018      3
020330800012202019      4
02035150005212008       2
0304302000092022        3
                     ... 
7801506222018           4
7801507472021           3
78016042212016          9
7801605692021           4
7801702372020           2
Length: 234, dtype: int64

In [351]:
from natasha import AddrExtractor, MorphVocab
addr_extractor = AddrExtractor(MorphVocab())

for num in join_df.groupby(['number']).apply(lambda df: df['objectid'].nunique()).sort_values()[-239:].index:
    print(num,
        join_df[join_df['number'] == num]['address_region_name'].unique(), join_df[join_df['number'] == num]['commissioningPlanDate'].unique(),
          join_df[join_df['number'] == num]['address_streetName'].unique(),
            #pd.Series(join_df[join_df['number'] == num]['Адрес объекта недвижимости (без номера дома/корпуса)'].unique()).map(lambda x: addr_extractor.find(x)),
         join_df[join_df['number'] == num]['livingArea'].nunique(), join_df[join_df['number'] == num]['latitude'].nunique(),
         join_df[join_df['number'] == num]['Номер объекта (дома, корпуса, литеры и тп)'].nunique())

723042032021 ['Тюменская область'] ['2023-12-31'] ['Николая Машарова'] 2 2 3
36363020000692017 ['Воронежская область'] ['2021-03-31' '2021-06-30'] ['Острогожская'] 2 2 1
36363020000572019 ['Воронежская область'] ['2021-12-31'] ['Крымская, Конструкторов, Пирогова'] 2 2 1
36363020000172021 ['Воронежская область'] ['2022-09-30'] ['Крымская, Конструкторов'] 2 1 2
57301000702018 ['Орловская область'] ['2023-03-31' '2023-12-31'] ['Родзевича-Белевича'] 2 2 3
63507312272019 ['Самарская область'] ['2022-12-31'] [None] 2 2 5
2330983892021 ['Краснодарский край'] ['2024-12-31'] ['Ленина'] 1 1 4
635073052082019 ['Самарская область'] ['2023-03-31'] [None] 2 2 9
69693040001292018 ['Тверская область'] ['2021-12-31' '2023-03-31'] ['Гусева'] 2 2 2
72304342021 ['Тюменская область'] ['2023-03-31'] ['Республики'] 2 1 1
2104122020 ['Чувашская Республика'] ['2020-12-31' '2021-06-30'] ['Новая'] 2 1 2
2104112019 ['Чувашская Республика'] ['2020-12-31' '2020-09-30'] ['Комсомольская'] 2 1 4
3630200064 ['Воронежск

In [324]:
join_df[join_df['number'] == num]['Номер объекта (дома, корпуса, литеры и тп)'].unique()

array(['-', 'Дом: 4, Объект: 1', 'Дом:4,Объект:1 [id1487]'], dtype=object)

In [328]:
for addr in join_df[join_df['number'] == num]['Номер объекта (дома, корпуса, литеры и тп)'].unique()[:40]:
    print(get_addres_df(addr, addr_extractor))

None
None
None
None
None
None


In [341]:
join_df.groupby(['number']).apply(lambda df: df['objectid'].nunique()).sort_values()[-239:].index

Index(['723042032021', '36363020000692017', '36363020000572019',
       '36363020000172021', '57301000702018', '63507312272019',
       '2330983892021', '635073052082019', '69693040001292018', '72304342021',
       ...
       '403010000062022', '52523030002005001062019', '40301000012',
       '50520000278', '234352452018', '52523030002005001062020',
       '5252303000200500103012018', '52523030002005001062021', '234352392018',
       ''],
      dtype='object', name='number', length=239)

In [353]:
def get_addres_df(addrs, extractor):
    
    return pd.DataFrame([get_addres_dict(addr, extractor) for addr in addrs])
    



def get_addres_dict(addres, extractor):
    find = extractor.find(addres)
    if find is None:
        return {}
    
    parts = find.fact.parts
    #print([part.type for part in parts])
    parts_dict = {
        'улица': None,
        'дом': None
    }
    for part in parts:
        if part.type in parts_dict:
            parts_dict[part.type] = part.value
    return parts_dict


def join_addr(df, addres_cols, extractor):
    return  pd.concat([get_addres_df(df[col].values, extractor) for col in addres_cols], axis = 1)
join_addr(join_df[join_df['number'] == '6060302000522017'], ['Адрес объекта недвижимости (без номера дома/корпуса)',
                                                              'Номер объекта (дома, корпуса, литеры и тп)'], addr_extractor)

,улица,дом
0,Инженерная,132
1,Инженерная,132
2,Инженерная,132
3,Инженерная,132
4,Инженерная,132
...,...,...
461,Инженерная,132
462,Инженерная,132
463,Инженерная,132
464,Инженерная,132


In [338]:
addr['дом'].unique()

array(['4', '2', None], dtype=object)

In [312]:
pd.Series(join_df[join_df['number'] == num]['Адрес объекта недвижимости (без номера дома/корпуса)'].unique()).iloc[:1].map(lambda x: addr_extractor.find(x))[0].as_json#.fact.parts[0]

OrderedDict([('start', 22),
             ('stop', 110),
             ('fact', Addr(
                  parts=[AddrPart(
                       value='Тюменская',
                       type='область'
                   ),
                   AddrPart(
                       value='Тюмень',
                       type='город'
                   ),
                   AddrPart(
                       value='Николая Машарова',
                       type='улица'
                   ),
                   AddrPart(
                       value='4',
                       type='дом'
                   ),
                   AddrPart(
                       value='№ 44',
                       type='квартира'
                   )]
              ))])

In [303]:
dir(pd.Series(join_df[join_df['number'] == num]['Адрес объекта недвижимости (без номера дома/корпуса)'].unique()).iloc[:1].map(lambda x: addr_extractor.find(x))[0])

['__annotations__',
 '__attributes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_pretty_',
 'as_json',
 'fact',
 'from_json',
 'start',
 'stop']

In [365]:
join_df[join_df['objectid'].isin(join_df.groupby(['objectid']).apply(lambda df: df['number'].nunique()).sort_values()[-91:].index)]
#       (join_df['Дата открытия '] <= pd.to_datetime(join_df['commissioningPlanDate']))].groupby(['number']).apply(lambda df: df['objectid'].nunique())

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,numer,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
26,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,29284,,1415.0,275,1.172140e+09,Территория застройки в границах улиц имени Мар...,None,None,None,(№13 по генплану),None,Нижний Новгород,г,52,Нижегородская область,None,None,None,"обл Нижегородская, г Нижний Новгород, д. (№13 ...",52,Нижегородская область,56.274748,44.049443,5711.10,1,Строится,52-000458,2021-03-31,2018-06-26,Министерство строительства Нижегородской области
34,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,16224,,1415.0,275,1.172140e+09,Территория застройки в границах улиц имени Мар...,None,None,None,(№2 по генплану),None,Нижний Новгород,г,52,Нижегородская область,None,None,None,"обл Нижегородская, г Нижний Новгород, улица Ро...",52,Нижегородская область,56.276244,44.052012,9111.70,2,Сдан,52-000269,2020-06-30,2018-06-05,Министерство строительства Нижегородской области
83,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,30801,,NaN,2893,3.913580e+08,"Г. Калуга, ул. Василия Стригунова, д.13",None,None,None,95,None,Калуга,г,40,Калужская область,Верховая,улица,None,"обл Калужская, г Калуга, улица Верховая, д. 95...",40,Калужская область,54.501807,36.185119,2079.60,1,Строится,40-000173,2020-09-30,2015-01-20,Городская Управа города Калуги
85,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,33822,,1415.0,275,1.172140e+09,Территория застройки в границах улиц имени Мар...,None,None,None,(№15 по генплану

In [367]:
join_df.groupby(['objectid']).apply(lambda df: df['number'].nunique()).sort_values()[-91:]

objectid
34502    2
26152    2
30801    2
30923    2
40293    2
        ..
38491    3
38492    3
45139    3
31868    3
43388    4
Length: 91, dtype: int64

In [404]:
for obj in join_df.groupby(['objectid']).apply(lambda df: df['number'].nunique()).sort_values()[-91:].index:
    print(join_df[join_df['objectid'] == obj]['address_region_name'].unique(), join_df[join_df['objectid'] == obj]['commissioningPlanDate'].nunique(),
         join_df[join_df['objectid'] == obj]['livingArea'].unique(), join_df[join_df['objectid'] == obj]['number'].unique())

['Липецкая область'] 1 [10298.55] ['48427010002672019' '4842701000542020']
['Нижегородская область'] 1 [3566.8] ['' '5252303000200500103012018']
['Калужская область'] 1 [2079.6] ['' '40301000012']
['Калужская область'] 1 [2824.5] ['' '40301000012']
['Саха (Якутия) Республика '] 1 [5609.24] ['14301000922020' '1414301000922020']
['Саха (Якутия) Республика '] 1 [5609.24] ['14301000922020' '1414301000922020']
['Чувашская Республика'] 1 [6710.08] ['21011212020' '210122021']
['Чувашская Республика'] 1 [6710.08] ['21011202020' '210132021']
['Чувашская Республика'] 1 [6910.02] ['2124032020' '2124102021']
['Оренбургская область'] 1 [12281.15] ['563010002282019' '563010000092021']
['Новосибирская область'] 1 [17631.67] ['5454303000522021' '5454303000522022']
['Оренбургская область'] 1 [0.] ['563010002282019' '563010000092021']
['Оренбургская область'] 1 [0.] ['563010002282019' '563010000092021']
['Брянская область'] 1 [7944.73] ['3230135152021' '3230135142022']
['Смоленская область'] 1 [5850.18]

In [156]:
join_df['numer'][join_df['numer'].map(lambda x: x.endswith('2022') if type(x) == str else False)]

572               0244212022
573               0244212022
574               0244212022
575               0244212022
576               0244212022
                 ...        
754456    772160000199942022
754457    772160000199942022
754458    772160000199942022
754459    772160000199942022
754460    772160000199942022
Name: numer, Length: 79809, dtype: object

In [234]:
join_df['numer'][join_df['numer'].map(lambda x: len(re.findall(r'№', str(x))) if x else None) > 0].dropna()

587982    №40301000012
587983    №40301000012
587984    №40301000012
587985    №40301000012
587986    №40301000012
587987    №40301000012
587988    №40301000012
587989    №40301000012
Name: numer, dtype: object

In [92]:
join_df['number'][join_df['number'].map(lambda x: len(re.findall(r'/', str(x))) if x else None) > 0].dropna().map(lambda x: len(x.split('/')[-1])).reset_index().groupby(['number']).count()

,index
number,
0,817
1,1456
2,1631
3,80
4,1403
5,1306
6,990
7,6090
8,225


In [72]:
join_df[join_df['numer'].isna()][['numer', 'number']]

,numer,number
563634,NaN,2828302000082016
563635,NaN,2828302000082016
563636,NaN,73733602016
563637,NaN,715220001722017
563638,NaN,502368032016
...,...,...
611951,NaN,39393100007272021
611952,NaN,3333501101402021
611953,NaN,63302000112022
611954,NaN,919330800007722021


In [74]:
join_df.count()[:60]

Дата последнего изменения                                   563634
Дата открытия                                               563634
Плановая дата окончания                                     563634
Фактическая дата закрытия счета                             563634
Последняя дата пополнения счета                             563634
Последняя сумма пополнения счета                            563634
Общая сумма пополнений в последнюю дату пополнений счета    563634
Эскроу с использованием ипотечных средств ВТБ               563634
Номер кредитного договора (если ипотека ВТБ)                563634
Код Филиала                                                 563634
Код ТП                                                      563634
Статус счета                                                563634
Номер счета эскроу                                          563634
Тип счета                                                   563634
Общая сумма остатка по счету                                56

In [131]:
from natasha import AddrExtractor, MorphVocab
addr_extractor = AddrExtractor(MorphVocab())
addr_extractor.find(esc['addressString'][2])

Match(
    start=25,
    stop=72,
    fact=Addr(
        parts=[AddrPart(
             value='Первомайский',
             type='посёлок'
         ),
         AddrPart(
             value='Утилина',
             type='проспект'
         )]
    )
)

In [128]:
esc['addressString'][2]

'обл Тульская, Щекинский, рабочий поселок Первомайский, проспект  Утилина'

In [120]:
esc

,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
0,69,28-RU 28302000-08-2016,NaN,10,5.518260e+09,None,None,None,None,None,4,Благовещенск,город,28,Амурская область,"Василенко, ЗПУ-2, литер 4",улица,None,"обл Амурская, город Благовещенск, улица Василе...",28,Амурская область,50.297644,127.496756,5567.30,2,Сдан,28-000025,2018-12-31,2016-12-05,Администрация города Благовещенска Амурской об...
1,631,73-73-360-2016,NaN,552,6.034090e+09,None,None,None,None,82,None,Ульяновск,г,73,Ульяновская область,Радищева,улица,None,"обл Ульяновская, г Ульяновск, Ленинский, улица...",73,Ульяновская область,54.329557,48.402656,7077.38,2,Сдан,73-000196,2019-06-30,2016-08-08,Администрация города Ульяновска
2,1439,RU 71522000-172-2017,NaN,958,6.535508e+09,None,None,Щекинский,None,None,None,Первомайский,рабочий поселок,71,Тульская область,Утилина,проспект,None,"обл Тульская, Щекинский, рабочий поселок Перво...",71,Тульская область,54.038241,37.502366,NaN,2,Сдан,71-000004,2019-03-31,2017-10-20,Администрация муниципального образования Щекин...
3,1933,RU50-23-6803-2016,NaN,1416,1.464219e+09,None,None,Подольский,None,42,None,Лаговское,с/п,50,Московская область,None,None,None,"обл Московская, Подольский, с/п Лаговское, д. ...",50,Московская область,55.386476,37.626107,1167.28,1,Строится,50-000044,2020-09-30,2016-12-15,Министерство жилищной политики Московской области
4,2072,46-Ru46302000-3203-2016,NaN,797,6.003461e+09,None,None,None,None,None,None,Курск,г,46,Курская область,Союзная,ул,None,"обл Курская, г Курск, ул Союзная",46,Курская область,51.745294,36.249905,0.00,2,Сдан,46-000006,2018-06-30,2016-05-12,Комитет архитектуры и градостроительства город...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50183,45748,91-RU93308000-0772-2021,NaN,802,6.347000e+03,Ограниченной:с юго-запада-ул.Русская; северо-з...,None,None,None,None,None,Симферополь,г,91,Крым Республика,None,None,None,"Респ Крым, г Симферополь, Ограниченной:с юго-з...",91,Крым Республика,44.926569,34.102721,0.00,1,Строится,91-000397,2022-12-31,2021-05-19,МИНИСТЕРСТВО ЖИЛИЩНОЙ ПОЛИТИКИ И ГОСУДАРСТВЕНН...
50184,45755,78-013-0357.1-2016,NaN,239,4.294680e+08,I этап строительства (секции 1-16),None,None,None,17,None,Санкт-Петербург,г,78,Санкт-Петербург город,Ремесленная,улица,None,"г Санкт-Петербург, улица Ремесленная, д. 17, I...",78,Санкт-Петербург город,59.957159,30.266568,32655.00,1,Строится,78-001840,2023-12-31,2017-01-13,Служба государственного строительного надзора ...
50185,45893,RU 23306000-3485-Р,NaN,12008,4.752097e+09,None,None,None,None,17/1,Б,Краснодар,г,23,Краснодарский край,Крылатская,ул,None,"край Краснодарский, г Краснодар, ул Крылатская...",23,Краснодарский край,45.099002,38.916572,13909.88,1,Строится,23-002514,2024-12-31,2014-06-30,ДЕПАРТАМЕНТ АРХИТЕКТУРЫ И ГРАДОСТРОИТЕЛЬСТВА А...
50186,45918,04-302000-41-2020,NaN,1323,7.789000e+03,None,None,None,None,None,None,Улан-Удэ,г,3,Бурятия Республика,140Б,микрорайон,None,"Респ Бурятия, г Улан-Удэ, Октябрьский, микрора...",3,Бурятия Республика,51.798826,107.685303,0.00,1,Строится,03-000187,2022-09-30,2020-08-05,МУНИЦИПАЛЬНОЕ УЧРЕЖДЕНИЕ КОМИТЕТ ПО СТРОИТЕЛЬС...


In [412]:
df.assign(numer = df['Номер разрешения на строительство'].map(lambda x: re.sub(r'\s|-|[a-zA-Zа-яА-ЯёЁ]|/\d{1,3}\b|/|\(.*\)|\D', '', str(x)) if x is not None else None)).merge(
    esc.assign(number = esc['number'].map(lambda x: re.sub(r'\s|-|[a-zA-Zа-яА-яёЁ]|/\d{1,3}\b|/|\(.*\)|\B[\(:;,№].{5,}|\B["].{7,}|\D', '', x) if x is not None else None)), 
         left_on = 'numer', right_on = 'number', how = 'inner')

,Дата последнего изменения,Дата открытия,Плановая дата окончания,Фактическая дата закрытия счета,Последняя дата пополнения счета,Последняя сумма пополнения счета,Общая сумма пополнений в последнюю дату пополнений счета,Эскроу с использованием ипотечных средств ВТБ,Номер кредитного договора (если ипотека ВТБ),Код Филиала,Код ТП,Статус счета,Номер счета эскроу,Тип счета,Общая сумма остатка по счету,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Номер КС,ФИО/Наименование Депонента,ИНН депонента,Наименование получателя,Сегмент получателя,ОГРН получателя,ИНН получателя,БИК банка получателя,Расчетный счет получателя,Номер ДДУ,Дата ДДУ,Дата регистрации ДДУ,ДДУ с несколькими участниками долевого строительства,Адрес объекта недвижимости (без номера дома/корпуса),"Номер объекта (дома, корпуса, литеры и тп)","Площадь объекта ДДУ, м2",Номер разрешения на строительство,Дата выдачи разрешения на строительство,Общая сумма по ДДУ,в т.ч. собственные средства,в т.ч. ипотечные средства ВТБ,в т.ч. материнский капитал,в т.ч. субсидии по военной ипотеке,в т.ч. ипотека другого банка,Cрок оплаты суммы по ДДУ,Cумма оплаты по графику,Владелец счета,numer,objectid,number,complexId,developerId,groupId,address_additionalInfo,address_building,address_district,address_domain,address_house,address_letter,address_localityName,address_localityType,address_region_code,address_region_name,address_streetName,address_streetType,address_unit,addressString,buildRegion_code,buildRegion_name,latitude,longitude,livingArea,objectStatus_code,objectStatus_name,projectDeclaration_number,commissioningPlanDate,issuedate,issuedby
0,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3098,,NaN,1224,4.223742e+09,"Микрорайон Королевка, участок ""Старый карьер""",None,Смоленский,None,None,None,Смоленск,город,67,Смоленская область,None,улица,None,"обл Смоленская, Смоленский, город Смоленск, улица",67,Смоленская область,54.812742,32.012638,3490.06,2,Сдан,67-000009,2019-06-30,2017-07-10,Администрация города Смоленска
1,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,3727,,NaN,539,1.442112e+09,None,None,None,None,None,None,Ульяновск,город,73,Ульяновская область,None,None,None,"обл Ульяновская, город Ульяновск, Засвияжский",73,Ульяновская область,54.281909,48.261848,NaN,2,Сдан,73-000020,2018-09-30,2016-09-16,Администрация г.Ульяновска
2,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,6464,,NaN,1623,5.155790e+08,None,None,None,None,3,None,Калининград,Город,39,Калининградская область,Александра Невского,Улица,None,"Калининградская область, Город Калининград, Ле...",39,Калининградская область,54.760969,20.541307,2273.60,2,Сдан,39-000173,2019-06-30,2017-11-24,"Агентство по архитектуре, градостроению и перс..."
3,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,2019-07-30 00:00:00,-,0.0,-,-,0,3103,Закрыт,40824810831036000011,В рамках 214-ФЗ,0.0,0.0,0.0,0.0,0.0,0.0,-,Сухих Антон Олегович,-,-,-,-,-,-,-,-,-,-,Нет,-,-,-,NaN,-,0.0,0.0,0.0,0.0,0.0,0.0,-,-,ФЛ,,13368,,NaN,2028,2.974180e+09,None,None,None,None,None,None,Калининград,город,39,Калининградская область,Карташева,улица,None,"обл Калининградская, город Калининград, улица ...",39,Калининградская область,54.719601,20.373171,NaN,2,Сдан,39-00027